# Generation Notebook

## 2D Generation

### Utils functions

In [19]:
from typing import TypedDict
from ipythonblocks import BlockGrid, colors
from IPython.display import clear_output

import time

In [3]:
world = [
    [0,0,1,0,0],
    [0,1,1,0,0],
    [1,1,1,1,0],
    [0,0,1,1,0],
    [0,0,0,1,0]
    ]

In [4]:
def print_dict(d: dict) -> None:
    for key, value in d.items():
        print(f"{key} : {value}")

In [5]:
def display_world(world: list[list[int]]) -> None:
    
    clear_output()
    grid = BlockGrid(width=len(world), height=len(world[0]), fill=colors['WhiteSmoke'], block_size=30)
    
    for x in range(len(world)):
        for y in range(len(world[x])):
            if world[x][y] == 1: 
                grid[x, y].set_colors(0,0,0)
                            
            if world[x][y] == 2: 
                grid[x, y].set_colors(0,150,64)
    
    grid.show()

In [6]:
display_world(world)

,,,,
,,,,
,,,,
,,,,
,,,,


In [7]:
def get_close(world: list[list[int]], posset: set, pos: tuple[int], cell_type: int) -> set:
        
    for x in range(pos[0]-1, pos[0]+2):
        if x == -1 or x == pos[0] or x >= len(world[0]):
            pass
        else : 
            if world[x][pos[1]] == cell_type  and (x,pos[1]) not in posset: 
                posset.add((x,pos[1]))
                
    for y in range(pos[1]-1, pos[1]+2):
        if y == -1 or y == pos[1] or y >= len(world[0]):
            pass 
        else : 
            if world[pos[0]][y] == cell_type and (pos[0],y) not in posset:
                posset.add((pos[0],y))     
                
    return posset

In [8]:
def get_biome_size(world: list[list[int]], biome_id: int) -> tuple[set, int] : 
    
    posset = set()
    
    for line in range(len(world)):
        for column in range(len(world)): 
            get_close(world, posset, (line, column), biome_id)  
    
    return posset, len(posset)

### 2 Biomes generation

In [9]:
import random

In [38]:
preset = {
    0 : {
        0 : 40, 
        1 : 60,        
    }, 
    1 : {
        0 : 60, 
        1 : 40, 
    }
}

In [39]:
class Cell : 
    
    def __init__(self, pos: tuple, biome: int) -> None: 
        self.pos = pos 
        self.biome = biome
        self.influences = []
    
    def __repr__(self) -> str:
        return f'[{self.pos[0]},{self.pos[1]}]'

In [40]:
def generate_empty(size: int) -> list[list[Cell]] :
    
    world = list()
    
    for x in range(size): 
        world.append([])
        for y in range(size):
            world[x].append(Cell((x,y), 0))
            
    return world

In [41]:
def analyse_neighbor(world: list[list[Cell]], pos: tuple[int]) -> None :

    world[pos[0]][pos[1]].influences = []
    
    for x in range(pos[0]-2, pos[0]+3):
        for y in range(pos[1]-2, pos[1]+3) :
            if ((x,y) == pos) or ((x or y) <= -1) or (y >= (len(world[0]) - 1)) or ((x >= (len(world[0]) - 1))):
                pass 
            else :
                world[pos[0]][pos[1]].influences.append(world[x][y].biome)

In [42]:
def set_biome(world: list[list[Cell]], pos: tuple[int]) -> None :
    
    l_prob = list()
    
    for element in world[pos[0]][pos[1]].influences :
        
        rm_list = []
        for biome, prob in preset[element].items() :
            rm_list.extend([biome]*prob)
        
        l_prob.append(random.choice(rm_list))
        
    world[pos[0]][pos[1]].biome = int(round(sum(l_prob)/len(l_prob), 0))

In [43]:
def world_to_grid(world: list[list[Cell]]) -> None:
    
    l = list()
    
    for x in range(len(world)):
        l.append([])
        for y in range(len(world)):
            l[x].append(world[x][y].biome)
    
    display_world(l)

In [48]:
world = generate_empty(20)

for i in range(1):
    for x in range(len(world)) :
        for y in range(len(world)) :
            analyse_neighbor(world, (x,y))
            set_biome(world, (x,y))

    world_to_grid(world)
    time.sleep(0.8)
    

,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
